In [16]:
import pandas as pd
from datetime import datetime, date
from pathlib import Path
import pyarrow as pa

In [17]:
data = pd.read_csv('../../data.csv', dtype=dict(first_name=str, email_address=str), parse_dates=['date_of_visits'])

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uuid             999 non-null    object        
 1   first_name       999 non-null    object        
 2   last_name        999 non-null    object        
 3   email_address    999 non-null    object        
 4   number_of_vists  999 non-null    int64         
 5   time_spent       999 non-null    float64       
 6   amount_spent     999 non-null    float64       
 7   date_of_visits   999 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 62.6+ KB


In [19]:
'date_of_visits' in data.columns

True

In [20]:
data.rename(columns={'number_of_vists': 'number_of_visits'}, inplace=True)

In [21]:
data.head()

,uuid,first_name,last_name,email_address,number_of_visits,time_spent,amount_spent,date_of_visits
0,2d574d1a,Sandra,Travis,rodriguezjohn@example.net,24,4.16,42.5,2008-05-24
1,5bb7d18a,Monica,Kathryn,gloria73@example.net,65,4.16,31.0,2017-09-28
2,b254ced1,Jamie,Megan,broberts@example.org,24,4.08,30.0,1983-12-15
3,547d7108,Lisa,Peter,jacobsonleah@example.net,15,7.44,18.5,2005-02-17
4,2c7c0535,Michele,Dr.,harry71@example.com,17,5.28,22.5,1979-10-23


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   uuid              999 non-null    object        
 1   first_name        999 non-null    object        
 2   last_name         999 non-null    object        
 3   email_address     999 non-null    object        
 4   number_of_visits  999 non-null    int64         
 5   time_spent        999 non-null    float64       
 6   amount_spent      999 non-null    float64       
 7   date_of_visits    999 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 62.6+ KB


In [27]:
SCHEMA = pa.schema([
    ('uuid', pa.string()), 
    ('first_name', pa.string()),
    ('last_name', pa.string()),
    ('email_address', pa.string()),
    ('number_of_visits', pa.int8()),
    ('time_spent', pa.float64()),
    ('amount_spent', pa.float64()),
    ('date_of_visits', pa.date32()),
])

today = date.today().strftime('%Y/%m/%d')
print(today)
Path("YEAR=<{}>/MONTH=<{}>/DAY=<{}>".format(*today.split('/')))
path = Path("YEAR=<{}>/MONTH=<{}>/DAY=<{}>".format(*today.split('/')))
output_filename = 'business_data.parquet'
# path = Path('.') / str(today.year) / "{:02d}".format(today.month) / str(today.day)
path.mkdir(exist_ok=True, parents=True)
# key = "YEAR=<{}>/MONTH=<{}>/DAY=<{}>/{}".format(*today.split('/'), output_filename)
data.to_parquet(path / output_filename, schema=SCHEMA, index=False)

str(path / output_filename)

2022/06/22


'YEAR=<2022>/MONTH=<06>/DAY=<22>/business_data.parquet'

In [ ]:
pq = pd.read_parquet(f'./{str(path / output_filename)}')
pq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uuid              999 non-null    object 
 1   first_name        999 non-null    object 
 2   last_name         999 non-null    object 
 3   email_address     999 non-null    object 
 4   number_of_visits  999 non-null    int8   
 5   time_spent        999 non-null    float64
 6   amount_spent      999 non-null    float64
 7   date_of_visits    999 non-null    object 
dtypes: float64(2), int8(1), object(5)
memory usage: 55.7+ KB


In [ ]:
pq.head()

NameError: name 'pq' is not defined

In [ ]:
# users = pq.loc[:, ['uuid', 'first_name', 'last_name', 'email_address']]
# visits = pq.loc[:, ['uuid', 'number_of_vists', 'time_spent', 'amount_spent', 'date_of_visits']]

In [ ]:
# visits

In [ ]:
# users

In [ ]:
import psycopg2

# conn = psycopg2.connect(
#     database="dev",
#     user="oluwasayo",
#     password="Oluwas@yo01",
#     host="poc-redshift-cluster-intern.cx2v4bbp8ahg.us-east-1.redshift.amazonaws.com",
#     port="5439"
# )
conn = psycopg2.connect(
    dbname="dev",
    user="awsuser",
    password="Password_01",
    host="redshift-poc-cluster.cy0d8fidtclf.us-east-1.redshift.amazonaws.com",
    port="5439"
)
cursor = conn.cursor()
cursor.execute('SELECT * FROM "dev"."public"."business_data";')
print(cursor.fetchall())
# cursor.execute()
# print(conn)

[('2d574d1a', 'Sandra', 'Travis', 'rodriguezjohn@example.net', 24, 4.16, 42.5, datetime.date(2008, 5, 24)), ('b254ced1', 'Jamie', 'Megan', 'broberts@example.org', 24, 4.08, 30.0, datetime.date(1983, 12, 15)), ('2c7c0535', 'Michele', 'Dr.', 'harry71@example.com', 17, 5.28, 22.5, datetime.date(1979, 10, 23)), ('e48fde05', 'Brian', 'Lori', 'thomasrichard@example.net', 54, 2.48, 27.0, datetime.date(2020, 5, 21)), ('6fbe9be5', 'Derek', 'James', 'coxjames@example.org', 0, 6.8, 41.5, datetime.date(1979, 4, 14)), ('c432214c', 'Carla', 'Lisa', 'tramirez@example.net', 46, 4.64, 21.0, datetime.date(2015, 12, 25)), ('efd3477f', 'Patty', 'Olivia', 'garcher@example.com', 99, 6.88, 33.0, datetime.date(1980, 7, 13)), ('d3cde392', 'Tommy', 'Joseph', 'ocrawford@example.org', 18, 6.0, 0.5, datetime.date(2012, 10, 1)), ('587f0913', 'Judy', 'Amanda', 'lgregory@example.com', 100, 5.04, 18.5, datetime.date(2018, 12, 9)), ('c23a3b0f', 'Stephanie', 'Steve', 'guerreroapril@example.com', 35, 6.48, 19.5, datetime

In [ ]:
pa.Schema.from_pandas(pq)


uuid: string
first_name: string
last_name: string
email_address: string
number_of_visits: int8
time_spent: double
amount_spent: double
date_of_visits: date32[day]
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1230